### This script is taking all the chapter abstract text, process it, and then call paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [2]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re

In [2]:
class GABS_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_abs):
        phrases_list = []
        self.phrases_abs = phrases_abs
        for text in self.phrases_abs:
            abs_ = parser.preprocess_cabs_abs(text)
            abs_paraphrase = augmentation.paraphrased_text(abs_)
            abs_list = [abs_paraphrase]
            phrases_list.append(abs_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_abs(self, abs_text):
        list_abs = []
        self.abs_text = abs_text
        abs_text_wrap = augmentation.wrap_text(self.abs_text)
        abs_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSummary\n" + abs_text_wrap
        list_abs.append(abs_title)
        
        return list_abs
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        font = ImageFont.truetype('NimbusMonoPS-Regular.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 14 width and height 8; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16
        abstract -- 16 width and height 12
        chapter abstract -- 18 width and height 6
        '''
        draw.text(((self.W-w)/13,(self.H-h)/16), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def abs_aug(self, abs_text_title):
        self.abs_text_title = abs_text_title
        for n, row in enumerate(self.abs_text_title):
            fileName = ("/home/mchou001/Label-GeneralAbstract/aug_text{}.png".format(n))
            ## dedication: 43, ack: 40, chapter abstract: 39, abstract - 42, gabs -- 38
            save_image = self.text_on_img(fileName, row, 38) 
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            iaa.Affine(rotate=(-6, 6)),
            iaa.AdditiveGaussianNoise(scale=(10, 50)),
            iaa.SaltAndPepper(p=0.1),
            iaa.GaussianBlur(sigma=0.5),
            iaa.LinearContrast(alpha=1),
            iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        lower = 0
        upper = 1000
        while lower < upper:
            for n, images in enumerate(self.img_files[0:]):
                aug_image = img_augmentation(image=cv2.imread(images))
                cv2.imwrite('./Label-GeneralAbstract/505/aug{}.png'.format(lower),aug_image)
                lower += 1

In [3]:
if __name__ == "__main__":
    
    parser = Preprocessor()
    augmentation = Augmentation()
    
    abs_text = GABS_TEXT_IMG_AUG()
    
    etd_data = pd.read_csv('ETD_aug.csv')
    etd_data.set_index("class", inplace = True)

    label_abs = etd_data.loc["Label-GeneralAbstract"]
    phrases_abs = label_abs['text']

    list_ = []
    for texts in phrases_abs:
        gabs =ast.literal_eval(texts)
        list_.extend(gabs)
    
    splitter = SentenceSplitter(language='en')
    gabs_list = splitter.split(str(list_))
    aug_gabs = augmentation.paraphrased_text(gabs_list)
    
    aug_gabs_list = abs_text.list_of_abs(aug_gabs)
    
    abs_text_aug = abs_text.abs_aug(aug_gabs_list)
    
    
    img_path = sorted(glob.glob('./Label-GeneralAbstract/*.png'), key = parser.numericalSort)
    
    augmented_img = abs_text.img_transform(img_path)
    
    %%time 
    [ x**2 for x in range(10000)]

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 21.5 µs


In [1]:
import os

# folder path
dir_path = './Label-GeneralAbstract/505'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 1000


In [3]:
#!zip -r gabs_txt_img.zip ./Label-GeneralAbstract/505/

In [4]:
parser = Preprocessor()
image_path = sorted(glob.glob('/home/mchou001/Label-GeneralAbstract/505/*.png'), key = parser.numericalSort)

df = pd.DataFrame(image_path, columns = ['filename'])
df['class'] = np.nan
df = df.fillna("Label-GeneralAbstract")
df.to_csv('gabs_img_aug.csv', encoding = 'utf-8', index= None)